In [1]:
import pandas as pd
import requests
import json
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import numpy as np

In [2]:
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

In [3]:
data = pd.read_csv("../Mobile/mobile_preproccessed_whole_dataset.csv")

In [4]:
data

,Unnamed: 0,Bug ID,Type,Summary,Product,Component,Status,Resolution,Updated,Bug Id,Description,Duplicate_Bug_Ids
0,0,1668376,defect,Test failure in dom/base/test/test_window_clos...,GeckoView,General,RESOLVED,FIXED,2020-10-14 03:01:38,1668376,the symptom was that the broadcastchannel me...,NaN
1,1,1631754,defect,PanZoomController erroneously returns INPUT_RE...,GeckoView,General,RESOLVED,FIXED,2020-05-23 04:47:46,1631754,for websites which have their own event handle...,NaN
2,2,1677838,defect,NumberFormatExceptionjava.lang.Integer in pars...,GeckoView,General,RESOLVED,FIXED,2021-01-06 13:28:48,1677838,we are seeing these crash being reported on se...,NaN
3,4,1686100,enhancement,Route Service.onTrimMemory notifications throu...,GeckoView,General,RESOLVED,FIXED,2021-01-12 02:17:23,1686100,since android may send these to the services h...,NaN
4,5,1630229,defect,Crash in [@ java.lang.AssertionError: at org.m...,GeckoView,General,RESOLVED,FIXED,2020-05-28 05:17:09,1630229,this bug is for crash report bp ebbf ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5315,5499,1683643,defect,Intermittent org.mozilla.geckoview.test.Select...,GeckoView,General,RESOLVED,DUPLICATE,2021-07-30 19:04:17,1683643,filed by rmaries at mozilla com parse...,1682775.0
5316,5500,1649160,defect,Download Feature: unicode RTLO char can fake t...,Firefox for iOS,General,RESOLVED,FIXED,2020-12-18 17:20:48,1649160,created attachment regedt‮gpj exe summary ...,NaN
5317,5501,1514619,defect,Menu options are in different order than new t...,Firefox for iOS,Menu and Toolbar,RESOLVED,INVALID,2019-02-11 16:36:05,1514619,created attachment png the hostor...,NaN
5318,5502,1546822,defect,Notifications repeating if not acted on in iOS...,Firefox for iOS,Sync,RESOLVED,WORKSFORME,2019-09-18 10:10:20,1546822,created attachment syncnotificationecho png ...,NaN


In [5]:
data = data.drop(columns=['Unnamed: 0'])
data

,Bug ID,Type,Summary,Product,Component,Status,Resolution,Updated,Bug Id,Description,Duplicate_Bug_Ids
0,1668376,defect,Test failure in dom/base/test/test_window_clos...,GeckoView,General,RESOLVED,FIXED,2020-10-14 03:01:38,1668376,the symptom was that the broadcastchannel me...,NaN
1,1631754,defect,PanZoomController erroneously returns INPUT_RE...,GeckoView,General,RESOLVED,FIXED,2020-05-23 04:47:46,1631754,for websites which have their own event handle...,NaN
2,1677838,defect,NumberFormatExceptionjava.lang.Integer in pars...,GeckoView,General,RESOLVED,FIXED,2021-01-06 13:28:48,1677838,we are seeing these crash being reported on se...,NaN
3,1686100,enhancement,Route Service.onTrimMemory notifications throu...,GeckoView,General,RESOLVED,FIXED,2021-01-12 02:17:23,1686100,since android may send these to the services h...,NaN
4,1630229,defect,Crash in [@ java.lang.AssertionError: at org.m...,GeckoView,General,RESOLVED,FIXED,2020-05-28 05:17:09,1630229,this bug is for crash report bp ebbf ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5315,1683643,defect,Intermittent org.mozilla.geckoview.test.Select...,GeckoView,General,RESOLVED,DUPLICATE,2021-07-30 19:04:17,1683643,filed by rmaries at mozilla com parse...,1682775.0
5316,1649160,defect,Download Feature: unicode RTLO char can fake t...,Firefox for iOS,General,RESOLVED,FIXED,2020-12-18 17:20:48,1649160,created attachment regedt‮gpj exe summary ...,NaN
5317,1514619,defect,Menu options are in different order than new t...,Firefox for iOS,Menu and Toolbar,RESOLVED,INVALID,2019-02-11 16:36:05,1514619,created attachment png the hostor...,NaN
5318,1546822,defect,Notifications repeating if not acted on in iOS...,Firefox for iOS,Sync,RESOLVED,WORKSFORME,2019-09-18 10:10:20,1546822,created attachment syncnotificationecho png ...,NaN


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5320 entries, 0 to 5319
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Bug ID             5320 non-null   int64  
 1   Type               5320 non-null   object 
 2   Summary            5320 non-null   object 
 3   Product            5320 non-null   object 
 4   Component          5320 non-null   object 
 5   Status             5320 non-null   object 
 6   Resolution         5320 non-null   object 
 7   Updated            5320 non-null   object 
 8   Bug Id             5320 non-null   int64  
 9   Description        5320 non-null   object 
 10  Duplicate_Bug_Ids  562 non-null    float64
dtypes: float64(1), int64(2), object(8)
memory usage: 457.3+ KB


In [7]:
sentences = data['Description']

In [8]:
tokens = []
for s in sentences:
    cleanedTex=re.sub(r'[^\w\s]','',s).lower()
    words = (word_tokenize(cleanedTex))
    tokens .append(words)

In [9]:
bm25 = BM25Okapi(tokens)

In [10]:
data = data.rename(columns = {"Bug ID":"Bug_Id"})

In [11]:
data

,Bug_Id,Type,Summary,Product,Component,Status,Resolution,Updated,Bug Id,Description,Duplicate_Bug_Ids
0,1668376,defect,Test failure in dom/base/test/test_window_clos...,GeckoView,General,RESOLVED,FIXED,2020-10-14 03:01:38,1668376,the symptom was that the broadcastchannel me...,NaN
1,1631754,defect,PanZoomController erroneously returns INPUT_RE...,GeckoView,General,RESOLVED,FIXED,2020-05-23 04:47:46,1631754,for websites which have their own event handle...,NaN
2,1677838,defect,NumberFormatExceptionjava.lang.Integer in pars...,GeckoView,General,RESOLVED,FIXED,2021-01-06 13:28:48,1677838,we are seeing these crash being reported on se...,NaN
3,1686100,enhancement,Route Service.onTrimMemory notifications throu...,GeckoView,General,RESOLVED,FIXED,2021-01-12 02:17:23,1686100,since android may send these to the services h...,NaN
4,1630229,defect,Crash in [@ java.lang.AssertionError: at org.m...,GeckoView,General,RESOLVED,FIXED,2020-05-28 05:17:09,1630229,this bug is for crash report bp ebbf ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5315,1683643,defect,Intermittent org.mozilla.geckoview.test.Select...,GeckoView,General,RESOLVED,DUPLICATE,2021-07-30 19:04:17,1683643,filed by rmaries at mozilla com parse...,1682775.0
5316,1649160,defect,Download Feature: unicode RTLO char can fake t...,Firefox for iOS,General,RESOLVED,FIXED,2020-12-18 17:20:48,1649160,created attachment regedt‮gpj exe summary ...,NaN
5317,1514619,defect,Menu options are in different order than new t...,Firefox for iOS,Menu and Toolbar,RESOLVED,INVALID,2019-02-11 16:36:05,1514619,created attachment png the hostor...,NaN
5318,1546822,defect,Notifications repeating if not acted on in iOS...,Firefox for iOS,Sync,RESOLVED,WORKSFORME,2019-09-18 10:10:20,1546822,created attachment syncnotificationecho png ...,NaN


In [12]:
import numpy as np

In [13]:
count=0
dup_indices_with_bug_id = {}
similarity_scores_before = []
similarity_scores = {}
issues = data.Bug_Id

for i, row in data.iterrows():
    
    issue_id = row.Bug_Id
    dup_id = row.Duplicate_Bug_Ids
    
    if dup_id not in issues.values:
        count+=1
#         print(f'{i}. Duplicate ID {dup_id} not found in issue IDs')
        continue

    similarity_row = bm25.get_scores(tokens[i])
    argsort_similarity = np.argsort(similarity_row)[::-1][1:]
    similarity_scores_before = (np.sort(similarity_row)[::-1][1:])
#     similarity_scores[issue_id] = np.min(similarity_scores_before[np.nonzero(similarity_scores_before)])
    dup_bug_index = list(issues[issues == dup_id].index)[0]
    dup_ranking = np.argmax(argsort_similarity == np.int64(dup_bug_index))
    
    dup_indices_with_bug_id[issue_id] = dup_ranking
# print(count)

In [14]:
count_at_1=0
count_at_5=0
count_at_10=0
count_at_20=0
count_at_25=0
count_at_30=0
count_at_50=0
count_at_75=0
count_at_100=0

for key, value in enumerate(dup_indices_with_bug_id.values()):
    if(value < 1):
        count_at_1+=1
    if(value < 5):
        count_at_5+=1
    if(value < 10):
        count_at_10+=1
    if(value < 20):
        count_at_20+=1
    if(value < 25):
        count_at_25+=1
    if(value < 30):
        count_at_30+=1
    if(value < 50):
        count_at_50+=1
    if(value < 75):
        count_at_75+=1
    if(value < 100):
        count_at_100+=1
        
print("count at 1",count_at_1)        
print("count at 5",count_at_5)
print("count at 10",count_at_10)
print("count at 20",count_at_20)
print("count at 25",count_at_25)
print("count at 30",count_at_30)
print("count at 50",count_at_50)
print("count at 75",count_at_75)
print("count at 100",count_at_100)

N = len(dup_indices_with_bug_id)

recall_rate_at_1 = count_at_1/N
print("recall_rate_at_1",recall_rate_at_1)

recall_rate_at_5 = count_at_5/N
print("recall_rate_at_5",recall_rate_at_5)

recall_rate_at_10 = count_at_10/N
print("recall_rate_at_10",recall_rate_at_10)

recall_rate_at_20 = count_at_20/N
print("recall_rate_at_20",recall_rate_at_20)

recall_rate_at_25 = count_at_25/N
print("recall_rate_at_25",recall_rate_at_25)

recall_rate_at_30 = count_at_30/N
print("recall_rate_at_30",recall_rate_at_30)

recall_rate_at_50 = count_at_50/N
print("recall_rate_at_50",recall_rate_at_50)

recall_rate_at_75 = count_at_75/N
print("recall_rate_at_75",recall_rate_at_75)

recall_rate_at_100 = count_at_100/N
print("recall_rate_at_10",recall_rate_at_100)

count at 1 49
count at 5 79
count at 10 90
count at 20 110
count at 25 117
count at 30 126
count at 50 147
count at 75 165
count at 100 173
recall_rate_at_1 0.14327485380116958
recall_rate_at_5 0.2309941520467836
recall_rate_at_10 0.2631578947368421
recall_rate_at_20 0.3216374269005848
recall_rate_at_25 0.34210526315789475
recall_rate_at_30 0.3684210526315789
recall_rate_at_50 0.4298245614035088
recall_rate_at_75 0.4824561403508772
recall_rate_at_10 0.5058479532163743


### Recall rate for the textually dissimilar data - BM25

In [15]:
textually_dissimilar_data = pd.read_csv("../Mobile/mobile_textually_dissimilar_duplicates.csv")

In [16]:
sentence_dissim = textually_dissimilar_data.Description

In [17]:
tokens_dissim = []
for s in sentence_dissim:
    cleanedTex=re.sub(r'[^\w\s]','',str(s)).lower()
    words = (word_tokenize(cleanedTex))
    tokens_dissim.append(words)

In [18]:
count=0
dup_indices_with_bug_id_dissimilar = {}
similarity_scores_before = []
similarity_scores = {}
issues = data.Bug_Id

for i, row in textually_dissimilar_data.iterrows():
    
    issue_id = row.Bug_Id
    dup_id = row.Duplicate_Bug_Ids
    
    if dup_id not in issues.values:
        count+=1
#         print(f'{i}. Duplicate ID {dup_id} not found in issue IDs')
        continue
    if i==4427:
        break
    similarity_row = bm25.get_scores(tokens_dissim[i])
    argsort_similarity = np.argsort(similarity_row)[::-1][1:]
    similarity_scores_before = (np.sort(similarity_row)[::-1][1:])
#     similarity_scores[issue_id] = np.min(similarity_scores_before[np.nonzero(similarity_scores_before)])
    dup_bug_index = list(issues[issues == dup_id].index)[0]
    dup_ranking = np.argmax(argsort_similarity == np.int64(dup_bug_index))
    
    dup_indices_with_bug_id_dissimilar[issue_id] = dup_ranking
# print(count)

In [19]:
count_at_1=0
count_at_5=0
count_at_10=0
count_at_20=0
count_at_25=0
count_at_30=0
count_at_50=0
count_at_75=0
count_at_100=0

for key, value in enumerate(dup_indices_with_bug_id_dissimilar.values()):
    if(value < 1):
        count_at_1+=1
    if(value < 5):
        count_at_5+=1
    if(value < 10):
        count_at_10+=1
    if(value < 20):
        count_at_20+=1
    if(value < 25):
        count_at_25+=1
    if(value < 30):
        count_at_30+=1
    if(value < 50):
        count_at_50+=1
    if(value < 75):
        count_at_75+=1
    if(value < 100):
        count_at_100+=1
        
print("count at 1",count_at_1)        
print("count at 5",count_at_5)
print("count at 10",count_at_10)
print("count at 20",count_at_20)
print("count at 25",count_at_25)
print("count at 30",count_at_30)
print("count at 50",count_at_50)
print("count at 75",count_at_75)
print("count at 100",count_at_100)

N = len(dup_indices_with_bug_id_dissimilar)

recall_rate_at_1 = count_at_1/N
print("recall_rate_at_1",recall_rate_at_1)

recall_rate_at_5 = count_at_5/N
print("recall_rate_at_5",recall_rate_at_5)

recall_rate_at_10 = count_at_10/N
print("recall_rate_at_10",recall_rate_at_10)

recall_rate_at_20 = count_at_20/N
print("recall_rate_at_20",recall_rate_at_20)

recall_rate_at_25 = count_at_25/N
print("recall_rate_at_25",recall_rate_at_25)

recall_rate_at_30 = count_at_30/N
print("recall_rate_at_30",recall_rate_at_30)

recall_rate_at_50 = count_at_50/N
print("recall_rate_at_50",recall_rate_at_50)

recall_rate_at_75 = count_at_75/N
print("recall_rate_at_75",recall_rate_at_75)

recall_rate_at_100 = count_at_100/N
print("recall_rate_at_10",recall_rate_at_100)

count at 1 22
count at 5 37
count at 10 42
count at 20 53
count at 25 56
count at 30 60
count at 50 69
count at 75 78
count at 100 83
recall_rate_at_1 0.11398963730569948
recall_rate_at_5 0.19170984455958548
recall_rate_at_10 0.21761658031088082
recall_rate_at_20 0.27461139896373055
recall_rate_at_25 0.29015544041450775
recall_rate_at_30 0.31088082901554404
recall_rate_at_50 0.35751295336787564
recall_rate_at_75 0.40414507772020725
recall_rate_at_10 0.43005181347150256


### Recall rate for the textually similar data - BM25

In [20]:
textually_similar_data = pd.read_csv("../Mobile/mobile_textually_similar_duplicates.csv")

In [21]:
sentence_sim = textually_similar_data.Description

In [22]:
tokens_sim = []
for s in sentence_sim:
    cleanedTex=re.sub(r'[^\w\s]','',str(s)).lower()
    words = (word_tokenize(cleanedTex))
    tokens_sim.append(words)

In [23]:
count=0
dup_indices_with_bug_id_similar = {}
similarity_scores_before = []
similarity_scores = {}

issues = data.Bug_Id

for i, row in textually_similar_data.iterrows():
    
    issue_id = row.Bug_Id
    dup_id = row.Duplicate_Bug_Ids
    
    if dup_id not in issues.values:
        count+=1
#         print(f'{i}. Duplicate ID {dup_id} not found in issue IDs')
        continue
    if i==5014:
        break
    similarity_row = bm25.get_scores(tokens_sim[i])
    argsort_similarity = np.argsort(similarity_row)[::-1][1:]
    similarity_scores_before = (np.sort(similarity_row)[::-1][1:])
    similarity_scores[issue_id] = np.min(similarity_scores_before[np.nonzero(similarity_scores_before)])
    dup_bug_index = list(issues[issues == dup_id].index)[0]
    dup_ranking = np.argmax(argsort_similarity == np.int64(dup_bug_index))
    
    dup_indices_with_bug_id_similar[issue_id] = dup_ranking
# print(count)

In [24]:
count_at_1=0
count_at_5=0
count_at_10=0
count_at_20=0
count_at_25=0
count_at_30=0
count_at_50=0
count_at_75=0
count_at_100=0

for key, value in enumerate(dup_indices_with_bug_id_similar.values()):
    if(value < 1):
        count_at_1+=1
    if(value < 5):
        count_at_5+=1
    if(value < 10):
        count_at_10+=1
    if(value < 20):
        count_at_20+=1
    if(value < 25):
        count_at_25+=1
    if(value < 30):
        count_at_30+=1
    if(value < 50):
        count_at_50+=1
    if(value < 75):
        count_at_75+=1
    if(value < 100):
        count_at_100+=1
        
print("count at 1",count_at_1)        
print("count at 5",count_at_5)
print("count at 10",count_at_10)
print("count at 20",count_at_20)
print("count at 25",count_at_25)
print("count at 30",count_at_30)
print("count at 50",count_at_50)
print("count at 75",count_at_75)
print("count at 100",count_at_100)

N = len(dup_indices_with_bug_id_similar)

recall_rate_at_1 = count_at_1/N
print("recall_rate_at_1",recall_rate_at_1)

recall_rate_at_5 = count_at_5/N
print("recall_rate_at_5",recall_rate_at_5)

recall_rate_at_10 = count_at_10/N
print("recall_rate_at_10",recall_rate_at_10)

recall_rate_at_20 = count_at_20/N
print("recall_rate_at_20",recall_rate_at_20)

recall_rate_at_25 = count_at_25/N
print("recall_rate_at_25",recall_rate_at_25)

recall_rate_at_30 = count_at_30/N
print("recall_rate_at_30",recall_rate_at_30)

recall_rate_at_50 = count_at_50/N
print("recall_rate_at_50",recall_rate_at_50)

recall_rate_at_75 = count_at_75/N
print("recall_rate_at_75",recall_rate_at_75)

recall_rate_at_100 = count_at_100/N
print("recall_rate_at_10",recall_rate_at_100)

count at 1 27
count at 5 42
count at 10 48
count at 20 57
count at 25 61
count at 30 66
count at 50 78
count at 75 87
count at 100 90
recall_rate_at_1 0.18120805369127516
recall_rate_at_5 0.28187919463087246
recall_rate_at_10 0.3221476510067114
recall_rate_at_20 0.3825503355704698
recall_rate_at_25 0.40939597315436244
recall_rate_at_30 0.4429530201342282
recall_rate_at_50 0.5234899328859061
recall_rate_at_75 0.5838926174496645
recall_rate_at_10 0.6040268456375839
